As base template I used this tutorial https://docs.opencv.org/3.4.1/db/df8/tutorial_py_meanshift.html

In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import imageio

from matplotlib import animation, rc
from IPython.display import HTML, Video

In [2]:
GROUNDTRUTH_RECT_FILE_NAME = 'groundtruth_rect.txt'
PATH_TO_RESULT = './Visual/video_results/'

# return path to dataset images by its name
def get_path_to_data_set(data_set):
    return (
        './Visual/datasets/{}/img/'.format(data_set),
        './Visual/datasets/{}/{}'.format(data_set, GROUNDTRUTH_RECT_FILE_NAME),
    )

In [3]:
# function which takes dataset and method as arguments and builts a video as a result of object tracking
def check_tracker(setting, method='MeanShift'):
    
    # take settings
    is_meanshift = method == 'MeanShift'    
    data_set = setting['data_set']
    bins = setting['bins']
    filter_from = np.array(setting['filter_from'])
    filter_to = np.array(setting['filter_to'])
    
    path_to_frames, path_to_gt_rect_file = get_path_to_data_set(data_set)      # take pathes to the dataset
    
    frames = sorted(os.listdir(path_to_frames))                                # read and sort frames
    first_frame = cv2.imread(path_to_frames + frames[0])                       # take the first frame
    
    # take the track window from groundtruth file or from the settings
    if 'roi' in setting:
        track_window = tuple(setting['roi'])
    else:
        track_window = tuple([
            int(x) for x in open(path_to_gt_rect_file).\
                readline().\
                replace('\n', '').\
                split(',')
        ])
    
    x,y,w,h = track_window
    
    color = (255,0,0)    
    cv2.rectangle(first_frame, (x, y), (x+w,y+h), color, 3)
    
    
    # set up the ROI for tracking
    roi = first_frame[y:y+h, x:x+w]
    hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)    
    mask = cv2.inRange(hsv_roi, filter_from, filter_to)
    roi_hist = cv2.calcHist([hsv_roi],[0], mask, [bins], [filter_from[0], filter_to[0]])
    cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
    
    # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
    
    # create video writer
    ft="XVID"
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    height , width , layers =  first_frame.shape
    video = cv2.VideoWriter(PATH_TO_RESULT + method + '_' + data_set + '.mp4', fourcc, 24., (width,height))
    
    for i in range(0, len(frames)):
        frame = cv2.imread(path_to_frames + frames[i])
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[filter_from[0], filter_to[0]],1)
        
        if is_meanshift:
            # apply meanshift to get the new location        
            ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        else:
            ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # Draw it on image
        x,y,w,h = track_window
        
        img = np.copy(frame)
        img = cv2.rectangle(img, (x,y), (x+w,y+h), 255,2)
        
#         plt.figure()
#         plt.imshow(img)
#         plt.show()
        
        # add image to the video writer
        video.write(img)        

    
    # save video
    cv2.destroyAllWindows()
    video.release()


In [4]:
# all this settings were matched manually by checking histograms and results
settings = [
    {
        'data_set': 'Bolt',
        'filter_from': (0., 0., 0.),
        'filter_to': (25.,255.,36.),
        'bins': 180
    },
    {
        'data_set': 'Basketball',
        'filter_from': (30.,60.,50.),
        'filter_to': (90.,255.,180.),
        'bins': 3
    },
    {
        'data_set': 'Soccer',
        'filter_from': (0.,60.,32.),
        'filter_to': (25.,255.,255.),
        'bins': 180
    },
    {
        'data_set': 'Jump',
        'filter_from': (10.,60.,30.),
        'filter_to': (35.,255.,155.),
        'bins': 30
    },
    {
        'data_set': 'Box',
        'roi': [478,143,80,111],
        'filter_from': (0.,0.,0.),
        'filter_to': (25.,255.,36.),
        'bins': 10
    }
]

for s in settings:
    check_tracker(s, 'MeanShift')
    check_tracker(s, 'CamShift')